In [1]:
import  torch, os
import  numpy as np
import  scipy.stats
from    torch.utils.data import DataLoader
from    torch.optim import lr_scheduler
import  random, sys, pickle
from utils.dataloader import train_data_gen , test_data_gen
# from maml_meta import Meta
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from    torch import optim
from maml_learner import Learner
from    torch.nn import functional as F
from    copy import deepcopy
import json
import shutil
import torchvision.utils as vutils

In [2]:
with open('maml_configs/0704_5way1shot2distractor1gan.json') as json_file:
    args = json.load(json_file)
print(args)
# if os.path.exists('maml_runs/' + args["save_path"]):
#     shutil.rmtree('maml_runs/' + args["save_path"])
writer = SummaryWriter('maml_runs/' + args["save_path"])

{'epoch': 60000, 'n_way': 5, 'k_spt': 1, 'k_qry': 15, 'img_sz': 84, 'task_num': 5, 'img_c': 3, 'meta_lr': 0.001, 'update_lr': 0.01, 'update_step': 5, 'update_step_test': 10, 'loss': 'cross_entropy', 'min_learning_rate': 1e-15, 'number_of_training_steps_per_iter': 4, 'multi_step_loss_num_epochs': 15, 'spy_gan_num': 1, 'qry_gan_num': 5, 'num_distractor': 2, 'gan': 1, 'spy_distractor_num': 1, 'qry_distractor_num': 5, 'batch_for_gradient': 25, 'no_save': 0, 'learn_inner_lr': 0, 'create_graph': 0, 'msl': 0, 'single_fast_test': 0, 'consine_schedule': 0, 'save_path': '0704_5way1shot2distractor1gan'}


In [3]:
def mkdir_p(path):
        
    if not os.path.exists("maml/" + path):
        os.makedirs("maml/" + path)     

In [4]:
spt_size = args["k_spt"] * args["n_way"]
qry_size = args["k_qry"] * args["n_way"]

In [5]:
fm = 42
config = [
    ('conv2d', [fm, 3, 3, 3, 1, 0]),
    ('leakyrelu', [0.2,True]),
    ('bn', [fm]),
    ('max_pool2d', [2, 2, 0]),
    ('conv2d', [fm, fm, 3, 3, 1, 0]),
    ('leakyrelu', [0.2,True]),
    ('bn', [fm]),
    ('max_pool2d', [2, 2, 0]),
    ('conv2d', [fm, fm, 3, 3, 1, 0]),
    ('leakyrelu', [0.2,True]),
    ('bn', [fm]),
    ('max_pool2d', [2, 2, 0]),
    ('conv2d', [fm, fm, 3, 3, 1, 0]),
    ('leakyrelu', [0.2,True]),
    ('bn', [fm]),
    ('max_pool2d', [2, 1, 0]),
    ('flatten', []),
    ('linear', [6, fm * 5 * 5])
]

In [6]:
train_data_generator = train_data_gen(args)
test_data_generator = test_data_gen(args)

load datasets/BelgiumTSC
load complete time 0.4288804531097412
load datasets/ArTS
load complete time 0.398226261138916
load datasets/chinese_traffic_sign
load complete time 0.7527906894683838
load datasets/CVL
load complete time 0.5750961303710938
load datasets/FullJCNN2013
load complete time 0.2891354560852051
load datasets/logo_2k
load complete time 1.0230731964111328
load datasets/GTSRB
load complete time 0.09882092475891113
load datasets/DFG
load complete time 0.035915374755859375


In [7]:
# train_dataloader = DataLoader(train_data_generator, args["task_num"], shuffle=True, num_workers=1, pin_memory=True)

In [8]:
# 

# def weights_init(m):
#     classname = m.__class__.__name__
#     if classname.find('Conv') != -1:
#         nn.init.normal_(m.weight.data, 0.0, 0.02)
#     elif classname.find('BatchNorm') != -1:
#         nn.init.normal_(m.weight.data, 1.0, 0.02)
#         nn.init.constant_(m.bias.data, 0)

In [9]:
device = torch.device("cuda:0")
nz = 100
ngf = 64
ndf = 64
nc = 3

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 0, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 0, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)
if args["gan"]:
    netG = Generator().to(device)
    netG.load_state_dict(torch.load("GAN_save_models/generator/fixed/model_step_15.pt")["model_state_dict"])
    # netG.apply(weights_init)
    # summary(netG,(nz,1,1))

In [10]:
# class Discriminator(nn.Module):
#     def __init__(self):
#         super(Discriminator, self).__init__()
#         self.main = nn.Sequential(
#             # input is (nc) x 64 x 64
#             nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
#             nn.LeakyReLU(0.2, inplace=True),
#             # state size. (ndf) x 32 x 32
#             nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
#             nn.BatchNorm2d(ndf * 2),
#             nn.LeakyReLU(0.2, inplace=True),
#             # state size. (ndf*2) x 16 x 16
#             nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
#             nn.BatchNorm2d(ndf * 4),
#             nn.LeakyReLU(0.2, inplace=True),
#             # state size. (ndf*4) x 8 x 8
#             nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
#             nn.BatchNorm2d(ndf * 8),
#             nn.LeakyReLU(0.2, inplace=True),
#             # state size. (ndf*8) x 4 x 4
#             nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
#             nn.Conv2d(1, 1, 2, 1, 0, bias=False),
#             nn.Sigmoid()
#         )

#     def forward(self, input):
#         return self.main(input)
# netD = Discriminator().to(device)
# netD.load_state_dict(torch.load("GAN_save_models/discrim/model_step_15.pt")["model_state_dict"])
# # netD.apply(weights_init)

In [11]:
# # Initialize BCELoss function
# criterion = nn.BCELoss()
# beta1 = 0.5
# # Create batch of latent vectors that we will use to visualize
# #  the progression of the generator
# fixed_noise = torch.randn(50, nz, 1, 1, device=device)

# # Establish convention for real and fake labels during training
# real_label = 1.
# fake_label = 0.

# # Setup Adam optimizers for both G and D
# optimizerD = optim.Adam(netD.parameters(), lr=0.0002, betas=(beta1, 0.999))
# optimizerG = optim.Adam(netG.parameters(), lr=0.0002, betas=(beta1, 0.999))

In [12]:
# # Training Loop
# import cProfile
# import pstats
# # Lists to keep track of progress
# img_list = []
# G_losses = []
# D_losses = []
# iters = 0
# num_epochs = 6000
# print("Starting Training Loop...")
#         # fetch meta_batchsz num of episode each time
# step = 0
# for epoch in range(args["epoch"]//6000):
#     # For each batch in the dataloader
#     train_dataloader = DataLoader(train_data_generator, args["task_num"], shuffle=True, num_workers=4, pin_memory=True)
#     for i, data in enumerate(train_dataloader, 0):
#         ############################
#         # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
#         ###########################
#         ## Train with all-real batch
#         netD.zero_grad()
#         # Format batch
#         real_cpu = data[0].to(device)
#         real_cpu = real_cpu.view(real_cpu.size(0)*real_cpu.size(1),real_cpu.size(2),real_cpu.size(3),real_cpu.size(4))
#         b_size = real_cpu.size(0)
#         label = torch.full((b_size,), real_label, dtype=torch.float, device=device)


#         # Forward pass real batch through D
#         output = netD(real_cpu).view(-1)
#         # Calculate loss on all-real batch

#         errD_real = criterion(output, label)
#         # Calculate gradients for D in backward pass
#         errD_real.backward()
#         D_x = output.mean().item()

#         ## Train with all-fake batch
#         # Generate batch of latent vectors
#         noise = torch.randn(b_size, nz, 1, 1, device=device)
#         # Generate fake image batch with G
#         fake = netG(noise)
#         label.fill_(fake_label)
#         # Classify all fake batch with D
#         output = netD(fake.detach()).view(-1)
#         # Calculate D's loss on the all-fake batch
#         errD_fake = criterion(output, label)
#         # Calculate the gradients for this batch, accumulated (summed) with previous gradients
#         errD_fake.backward()
#         D_G_z1 = output.mean().item()
#         # Compute error of D as sum over the fake and the real batches
#         errD = errD_real + errD_fake
#         # Update D
#         optimizerD.step()

#         ############################
#         # (2) Update G network: maximize log(D(G(z)))
#         ###########################
#         netG.zero_grad()
#         label.fill_(real_label)  # fake labels are real for generator cost
#         # Since we just updated D, perform another forward pass of all-fake batch through D
#         output = netD(fake).view(-1)
#         # Calculate G's loss based on this output
#         errG = criterion(output, label)
#         # Calculate gradients for G
#         errG.backward()
#         D_G_z2 = output.mean().item()
#         # Update G
#         optimizerG.step()
#         # Output training stats
#         if i % 100 == 0:
#             print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
#                   % (epoch, args["epoch"]//6000, i, len(train_dataloader),
#                      errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

#         # Save Losses for plotting later
#         G_losses.append(errG.item())
#         D_losses.append(errD.item())

#         # Check how the generator is doing by saving G's output on fixed_noise
#         if (i % 500 == 0) or ((epoch == num_epochs-1) and (i == len(train_dataloader)-1)):
#             with torch.no_grad():
#                 fake = netG(fixed_noise).detach().cpu()
#             img_list.append(vutils.make_grid(fake, padding=2, normalize=True))
#         iters += 1

#     torch.save({'model_state_dict': netD.state_dict()}, "GAN_save_models/discrim/" + "model_step_" + str(epoch) + ".pt")
#     torch.save({'model_state_dict': netG.state_dict()}, "GAN_save_models/generator/" + "model_step_" + str(epoch) + ".pt")


In [13]:
class Meta(nn.Module):
    """
    Meta Learner
    """
    def __init__(self, args, config):
        """

        :param args:
        """
        super(Meta, self).__init__()

        self.update_lr = args["update_lr"]
        self.meta_lr = args["meta_lr"]
        self.n_way = args["n_way"]
        self.k_spt = args["k_spt"]
        self.k_qry = args["k_qry"]
        self.task_num = args["task_num"]
        self.update_step = args["update_step"]
        self.update_step_test = args["update_step_test"]
        self.distractor = args["num_distractor"]
        self.net = Learner(config, args["img_c"], args["img_sz"])
        self.meta_optim = optim.Adam(self.net.parameters(), lr=self.meta_lr)
        self.device = torch.device('cuda')
        self.gan = args["gan"]
    def forward(self, x_spt, y_spt, x_qry, y_qry, unlabel_spt_image=None, unlabel_qry_image=None,gan_spt=None, gan_qry=None):
        """

        :param x_spt:   [b, setsz, c_, h, w]
        :param y_spt:   [b, setsz]
        :param x_qry:   [b, querysz, c_, h, w]
        :param y_qry:   [b, querysz]
        :return:
        """
        task_num, setsz, c_, h, w = x_spt.size()
        querysz = x_qry.size(1)
        if self.gan :
            gan_sptsz = gan_spt.size(1)
            gan_qrysz = gan_qry.size(1)
        
        else:
            gan_qrysz = 0
        if self.distractor or gan_sptsz:
            corrects = {}
            corrects["total_query_nway"] = np.zeros(self.update_step + 1)
            if self.distractor:
                unlabel_querysz = unlabel_qry.size(1)
                corrects["query_nway"] = np.zeros(self.update_step + 1)
                corrects["label_query_nway"] = np.zeros(self.update_step + 1)
                corrects["unlabel_query_nway"] = np.zeros(self.update_step + 1)
            if self.gan :
                corrects["gan_query_nway"] = np.zeros(self.update_step + 1)
        else:
            corrects = {key: np.zeros(self.update_step + 1) for key in 
                [
                "query_nway"
                ]}
        losses_q = [0 for _ in range(self.update_step + 1)]  # losses_q[i] is the loss on step i

        for i in range(task_num):
            spt_image = x_spt[i]
            spt_label = y_spt[i]
            qry_image = x_qry[i]
            qry_label = y_qry[i]
            if self.distractor:
                spt_image = torch.concat((spt_image,unlabel_spt[i]))
                spt_unlabel_label = torch.full((unlabel_spt.size(1),), 5, dtype=torch.long,device=self.device)
                spt_label = torch.cat((spt_label,spt_unlabel_label))
                qry_image = torch.concat((qry_image,unlabel_qry[i]))
                qry_unlabel_label = torch.full((unlabel_qry.size(1),), 5, dtype=torch.long,device=self.device)
                qry_label = torch.cat((qry_label,qry_unlabel_label))
            if self.gan :
                spt_image = torch.concat((spt_image,gan_spt[i]))
                spt_gan_label = torch.full((gan_spt.size(1),), 5, dtype=torch.long,device=self.device)
                spt_label = torch.cat((spt_label,spt_gan_label))
                qry_image = torch.concat((qry_image,gan_qry[i]))
                qry_gan_label = torch.full((gan_qry.size(1),), 5, dtype=torch.long,device=self.device)
                qry_label = torch.cat((qry_label,qry_gan_label))

            # 1. run the i-th task and compute loss for k=0
            logits = self.net(spt_image, vars=None, bn_training=True)
            loss = F.cross_entropy(logits, spt_label)
            grad = torch.autograd.grad(loss, self.net.parameters())
            fast_weights = list(map(lambda p: p[1] - self.update_lr * p[0], zip(grad, self.net.parameters())))

            # this is the loss and accuracy before first update
            with torch.no_grad():
                # [setsz, nway]
                if self.distractor or gan_sptsz:
                    total_logits_q = self.net(qry_image, self.net.parameters(), bn_training=False)
                    total_pred_q = F.softmax(total_logits_q, dim=1).argmax(dim=1)
                    total_q_correct = torch.eq(total_pred_q, qry_label).sum().item()
                    corrects['total_query_nway'][0] += total_q_correct
                    loss_q = F.cross_entropy(total_logits_q, qry_label)
                    losses_q[0] += loss_q
                    if self.distractor:
                        label_logits_q = self.net(x_qry[i], self.net.parameters(), bn_training=False)
                        label_pred_q = F.softmax(label_logits_q, dim=1).argmax(dim=1)
                        label_pred_q_correct = torch.eq(label_pred_q, y_qry[i]).sum().item()
                        corrects['label_query_nway'][0] += label_pred_q_correct
                        label_pred_q = F.softmax(label_logits_q[:,:-1], dim=1).argmax(dim=1)
                        label_pred_q_correct = torch.eq(label_pred_q, y_qry[i]).sum().item()
                        corrects['query_nway'][0] += label_pred_q_correct
                        

                        unlabel_logits_q = self.net(unlabel_qry[i], self.net.parameters(), bn_training=False)
                        unlabel_pred_q = F.softmax(unlabel_logits_q, dim=1).argmax(dim=1)
                        other = torch.eq(unlabel_pred_q, qry_unlabel_label).sum().item()
                        corrects["unlabel_query_nway"][0] += other
                    if self.gan :
                        gan_logits_q = self.net(gan_qry[i], self.net.parameters(), bn_training=False)
                        gan_pred_q = F.softmax(gan_logits_q, dim=1).argmax(dim=1)
                        gan_counts = torch.eq(gan_pred_q, qry_gan_label).sum().item()
                        corrects["gan_query_nway"][0] += gan_counts
                else:
                    logits_q = self.net(qry_image, self.net.parameters(), bn_training=True)
                    loss_q = F.cross_entropy(logits_q, qry_label)
                    pred_q = F.softmax(logits_q, dim=1).argmax(dim=1)
                    q_discrim_correct = torch.eq(pred_q, qry_label).sum().item()
                    corrects['query_nway'][0] += q_discrim_correct
                    losses_q[0] += loss_q
            # this is the loss and accuracy after the first update
            with torch.no_grad():
                # [setsz, nway]
                if self.distractor or gan_sptsz:
                    
                    total_logits_q = self.net(qry_image,fast_weights , bn_training=False)
                    total_pred_q = F.softmax(total_logits_q, dim=1).argmax(dim=1)
                    total_q_correct = torch.eq(total_pred_q, qry_label).sum().item()
                    corrects['total_query_nway'][1] += total_q_correct
                    loss_q = F.cross_entropy(total_logits_q, qry_label)
                    losses_q[1] += loss_q
                    if self.distractor:
                        label_logits_q = self.net(x_qry[i], fast_weights, bn_training=False)
                        label_pred_q = F.softmax(label_logits_q, dim=1).argmax(dim=1)
                        label_pred_q_correct = torch.eq(label_pred_q, y_qry[i]).sum().item()
                        corrects['label_query_nway'][1] += label_pred_q_correct
                        label_pred_q = F.softmax(label_logits_q[:,:-1], dim=1).argmax(dim=1)
                        label_pred_q_correct = torch.eq(label_pred_q, y_qry[i]).sum().item()
                        corrects['query_nway'][1] += label_pred_q_correct
                        
                        unlabel_logits_q = self.net(unlabel_qry[i], fast_weights, bn_training=False)
                        unlabel_pred_q = F.softmax(unlabel_logits_q, dim=1).argmax(dim=1)
                        other = torch.eq(unlabel_pred_q, qry_unlabel_label).sum().item()
                        corrects["unlabel_query_nway"][1] += other
                    if self.gan :
                        gan_logits_q = self.net(gan_qry[i], fast_weights, bn_training=False)
                        gan_pred_q = F.softmax(gan_logits_q, dim=1).argmax(dim=1)
                        gan_counts = torch.eq(gan_pred_q, qry_gan_label).sum().item()
                        corrects["gan_query_nway"][1] += gan_counts
                else:
                    logits_q = self.net(qry_image, fast_weights, bn_training=False)
                    loss_q = F.cross_entropy(logits_q, qry_label)
                    pred_q = F.softmax(logits_q, dim=1).argmax(dim=1)
                    q_discrim_correct = torch.eq(pred_q, qry_label).sum().item()
                    corrects['query_nway'][1] += q_discrim_correct
                    losses_q[1] += loss_q
            for k in range(1, self.update_step):
                # 1. run the i-th task and compute loss for k=1~K-1
                logits = self.net(spt_image, fast_weights, bn_training=True)
                loss = F.cross_entropy(logits, spt_label)
                # 2. compute grad on theta_pi
                grad = torch.autograd.grad(loss, fast_weights)
                # 3. theta_pi = theta_pi - train_lr * grad
                fast_weights = list(map(lambda p: p[1] - self.update_lr * p[0], zip(grad, fast_weights)))

                logits_q = self.net(qry_image, fast_weights, bn_training=True)
                # loss_q will be overwritten and just keep the loss_q on last update step.
                loss_q = F.cross_entropy(logits_q, qry_label)
                losses_q[k + 1] += loss_q

                with torch.no_grad():
                    if self.distractor or gan_sptsz:
                        total_logits_q = self.net(qry_image, fast_weights, bn_training=False)
                        total_pred_q = F.softmax(total_logits_q, dim=1).argmax(dim=1)
                        total_q_correct = torch.eq(total_pred_q, qry_label).sum().item()
                        corrects['total_query_nway'][k+1] += total_q_correct
                        if self.distractor:
                            label_logits_q = self.net(x_qry[i], fast_weights, bn_training=False)
                            label_pred_q = F.softmax(label_logits_q, dim=1).argmax(dim=1)
                            label_pred_q_correct = torch.eq(label_pred_q, y_qry[i]).sum().item()
                            corrects['label_query_nway'][k+1] += label_pred_q_correct
                            label_pred_q = F.softmax(label_logits_q[:,:-1], dim=1).argmax(dim=1)
                            label_pred_q_correct = torch.eq(label_pred_q, y_qry[i]).sum().item()
                            corrects['query_nway'][k+1] += label_pred_q_correct
                            
                            unlabel_logits_q = self.net(unlabel_qry[i], fast_weights, bn_training=False)
                            unlabel_pred_q = F.softmax(unlabel_logits_q, dim=1).argmax(dim=1)
                            other = torch.eq(unlabel_pred_q, qry_unlabel_label).sum().item()
                            corrects["unlabel_query_nway"][k+1] += other
                        if self.gan :
                            gan_logits_q = self.net(gan_qry[i], fast_weights, bn_training=False)
                            gan_pred_q = F.softmax(gan_logits_q, dim=1).argmax(dim=1)
                            gan_counts = torch.eq(gan_pred_q, qry_gan_label).sum().item()
                            corrects["gan_query_nway"][k+1] += gan_counts
                    else:
                        pred_q = F.softmax(logits_q, dim=1).argmax(dim=1)
                        q_discrim_correct = torch.eq(pred_q, qry_label).sum().item()
                        corrects['query_nway'][k+1] += q_discrim_correct

        # end of all tasks
        # sum over all losses on query set across all tasks
        loss_q = losses_q[-1] / task_num
        # optimize theta parameters
        self.meta_optim.zero_grad()
        loss_q.backward()

        self.meta_optim.step()
        
        accs = {}
        if (self.distractor or self.gan):
            accs["total_query_nway"] = corrects["total_query_nway"] / (task_num * (querysz + unlabel_querysz + gan_qrysz))
            if self.distractor:
                accs["label_query_nway"] = corrects["label_query_nway"] / (task_num * querysz)
                accs["query_nway"] = corrects["query_nway"] / (task_num * querysz)
                accs["unlabel_query_nway"] = corrects["unlabel_query_nway"] / (task_num * unlabel_querysz)
            if gan_qrysz:
                accs['gan_query_nway'] = corrects["gan_query_nway"] / (task_num * gan_qrysz)
        else:
            accs["query_nway"] = corrects["query_nway"] / (task_num * querysz)
        return accs,loss_q


    def finetunning(self, x_spt, y_spt, x_qry, y_qry, unlabel_spt=None, unlabel_qry=None, gan_spt=None, gan_qry=None):

        assert len(x_spt.shape) == 4
        querysz = x_qry.size(0)
        if self.gan :
            gan_sptsz = gan_spt.size(0)
            gan_qrysz = gan_qry.size(0)
        else:
            gan_sptsz = gan_qrysz = 0
        if self.distractor or self.gan:
            corrects = {}
            corrects["total_query_nway"] = np.zeros(self.update_step_test + 1)
            if self.distractor:
                unlabel_querysz = unlabel_qry.size(0)
                corrects["label_query_nway"] = np.zeros(self.update_step_test + 1)
                corrects["query_nway"] = np.zeros(self.update_step_test + 1)
                corrects["unlabel_query_nway"] = np.zeros(self.update_step_test + 1)
            if self.gan:
                corrects["gan_query_nway"] = np.zeros(self.update_step_test + 1)
        else:
            corrects = {key: np.zeros(self.update_step_test + 1) for key in 
                            [
                            "query_nway"
                            ]}
        # in order to not ruin the state of running_mean/variance and bn_weight/bias
        # we finetunning on the copied model instead of self.net
        net = deepcopy(self.net)
        spt_image = x_spt
        spt_label = y_spt
        qry_image = x_qry
        qry_label = y_qry
        if self.distractor:
            spt_image = torch.concat((spt_image,unlabel_spt))
            spt_unlabel_label = torch.full((unlabel_spt.size(0),), 5, dtype=torch.long,device=self.device)
            spt_label = torch.cat((spt_label,spt_unlabel_label))
            qry_image = torch.concat((qry_image,unlabel_qry))
            qry_unlabel_label = torch.full((unlabel_qry.size(0),), 5, dtype=torch.long,device=self.device)
            qry_label = torch.cat((qry_label,qry_unlabel_label))
        if self.gan :
            spt_image = torch.concat((spt_image,gan_spt))
            spt_gan_label = torch.full((gan_spt.size(0),), 5, dtype=torch.long,device=self.device)
            spt_label = torch.cat((spt_label,spt_gan_label))
            qry_image = torch.concat((qry_image,gan_qry))
            qry_gan_label = torch.full((gan_qry.size(0),), 5, dtype=torch.long,device=self.device)
            qry_label = torch.cat((qry_label,qry_gan_label))
        
        # 1. run the i-th task and compute loss for k=0
        logits = net(spt_image)
        loss = F.cross_entropy(logits, spt_label)
        grad = torch.autograd.grad(loss, net.parameters())
        fast_weights = list(map(lambda p: p[1] - self.update_lr * p[0], zip(grad, net.parameters())))

        # this is the loss and accuracy before first update
        with torch.no_grad():
            if self.distractor or gan_sptsz:
                total_logits_q = self.net(qry_image, self.net.parameters(), bn_training=False)
                total_pred_q = F.softmax(total_logits_q, dim=1).argmax(dim=1)
                total_q_correct = torch.eq(total_pred_q, qry_label).sum().item()
                corrects['total_query_nway'][0] += total_q_correct
                loss_q = F.cross_entropy(total_logits_q, qry_label)
                if self.distractor:
                    label_logits_q = self.net(x_qry, self.net.parameters(), bn_training=False)
                    label_pred_q = F.softmax(label_logits_q, dim=1).argmax(dim=1)
                    label_pred_q_correct = torch.eq(label_pred_q, y_qry).sum().item()
                    corrects['label_query_nway'][0] += label_pred_q_correct
                    
                    label_logits_q = self.net(x_qry, self.net.parameters(), bn_training=False)
                    label_pred_q = F.softmax(label_logits_q[:,:-1], dim=1).argmax(dim=1)
                    label_pred_q_correct = torch.eq(label_pred_q, y_qry).sum().item()
                    corrects['query_nway'][0] += label_pred_q_correct
                    
                    unlabel_logits_q = self.net(unlabel_qry, self.net.parameters(), bn_training=False)
                    unlabel_pred_q = F.softmax(unlabel_logits_q, dim=1).argmax(dim=1)
                    other = torch.eq(unlabel_pred_q, qry_unlabel_label).sum().item()
                    corrects["unlabel_query_nway"][0] += other
                if self.gan :
                    gan_logits_q = self.net(gan_qry, self.net.parameters(), bn_training=False)
                    gan_pred_q = F.softmax(gan_logits_q, dim=1).argmax(dim=1)
                    gan_counts = torch.eq(gan_pred_q, qry_gan_label).sum().item()
                    corrects["gan_query_nway"][0] += gan_counts
            else:
                logits_q = self.net(qry_image, self.net.parameters(), bn_training=True)
                pred_q = F.softmax(logits_q, dim=1).argmax(dim=1)
                q_discrim_correct = torch.eq(pred_q, qry_label).sum().item()
                corrects['query_nway'][0] += q_discrim_correct
        # this is the loss and accuracy after the first update
        with torch.no_grad():
            if self.distractor or gan_sptsz:

                total_logits_q = self.net(qry_image,fast_weights , bn_training=False)
                total_pred_q = F.softmax(total_logits_q, dim=1).argmax(dim=1)
                total_q_correct = torch.eq(total_pred_q, qry_label).sum().item()
                corrects['total_query_nway'][1] += total_q_correct
                loss_q = F.cross_entropy(total_logits_q, qry_label)
                if self.distractor:
                    label_logits_q = self.net(x_qry, fast_weights, bn_training=False)
                    label_pred_q = F.softmax(label_logits_q, dim=1).argmax(dim=1)
                    label_pred_q_correct = torch.eq(label_pred_q, y_qry).sum().item()
                    corrects['label_query_nway'][1] += label_pred_q_correct
                    
                    label_logits_q = self.net(x_qry, fast_weights, bn_training=False)
                    label_pred_q = F.softmax(label_logits_q[:,:-1], dim=1).argmax(dim=1)
                    label_pred_q_correct = torch.eq(label_pred_q, y_qry).sum().item()
                    corrects['query_nway'][1] += label_pred_q_correct
                    
                    unlabel_logits_q = self.net(unlabel_qry, fast_weights, bn_training=False)
                    unlabel_pred_q = F.softmax(unlabel_logits_q, dim=1).argmax(dim=1)
                    other = torch.eq(unlabel_pred_q, qry_unlabel_label).sum().item()
                    corrects["unlabel_query_nway"][1] += other
                if self.gan :
                    gan_logits_q = self.net(gan_qry, fast_weights, bn_training=False)
                    gan_pred_q = F.softmax(gan_logits_q, dim=1).argmax(dim=1)
                    gan_counts = torch.eq(gan_pred_q, qry_gan_label).sum().item()
                    corrects["gan_query_nway"][1] += gan_counts
            else:
                logits_q = self.net(qry_image, fast_weights, bn_training=True)
                pred_q = F.softmax(logits_q, dim=1).argmax(dim=1)
                q_discrim_correct = torch.eq(pred_q, qry_label).sum().item()
                corrects['query_nway'][1] += q_discrim_correct
        for k in range(1, self.update_step_test):
            # 1. run the i-th task and compute loss for k=1~K-1
            logits = net(spt_image, fast_weights, bn_training=True)
            loss = F.cross_entropy(logits, spt_label)
            # 2. compute grad on theta_pi
            grad = torch.autograd.grad(loss, fast_weights)
            # 3. theta_pi = theta_pi - train_lr * grad
            fast_weights = list(map(lambda p: p[1] - self.update_lr * p[0], zip(grad, fast_weights)))

            logits_q = net(qry_image, fast_weights, bn_training=True)
            # loss_q will be overwritten and just keep the loss_q on last update step.
            loss_q = F.cross_entropy(logits_q, qry_label)

            with torch.no_grad():
                if self.distractor or gan_sptsz:
                    total_logits_q = self.net(qry_image, fast_weights, bn_training=False)
                    total_pred_q = F.softmax(total_logits_q, dim=1).argmax(dim=1)
                    total_q_correct = torch.eq(total_pred_q, qry_label).sum().item()
                    corrects['total_query_nway'][k+1] += total_q_correct
                    if self.distractor:
                        label_logits_q = self.net(x_qry, fast_weights, bn_training=False)
                        label_pred_q = F.softmax(label_logits_q, dim=1).argmax(dim=1)
                        label_pred_q_correct = torch.eq(label_pred_q, y_qry).sum().item()
                        corrects['label_query_nway'][k+1] += label_pred_q_correct
                        
                        label_logits_q = self.net(x_qry, fast_weights, bn_training=False)
                        label_pred_q = F.softmax(label_logits_q[:,:-1], dim=1).argmax(dim=1)
                        label_pred_q_correct = torch.eq(label_pred_q, y_qry).sum().item()
                        corrects['query_nway'][k+1] += label_pred_q_correct

                        unlabel_logits_q = self.net(unlabel_qry, fast_weights, bn_training=False)
                        unlabel_pred_q = F.softmax(unlabel_logits_q, dim=1).argmax(dim=1)
                        other = torch.eq(unlabel_pred_q, qry_unlabel_label).sum().item()
                        corrects["unlabel_query_nway"][k+1] += other
                    if self.gan :
                        gan_logits_q = self.net(gan_qry, fast_weights, bn_training=False)
                        gan_pred_q = F.softmax(gan_logits_q, dim=1).argmax(dim=1)
                        gan_counts = torch.eq(gan_pred_q, qry_gan_label).sum().item()
                        corrects["gan_query_nway"][k+1] += gan_counts
                else:
                    logits_q = self.net(qry_image, fast_weights, bn_training=True)
                    pred_q = F.softmax(logits_q, dim=1).argmax(dim=1)
                    q_discrim_correct = torch.eq(pred_q, qry_label).sum().item()
                    corrects['query_nway'][k+1] += q_discrim_correct
        del net
        accs = {}
        if (self.distractor or self.gan):
            accs["total_query_nway"] = corrects["total_query_nway"] / (querysz + unlabel_querysz + gan_qrysz)
            if self.distractor:
                accs["label_query_nway"] = corrects["label_query_nway"] / querysz
                accs["query_nway"] = corrects["query_nway"] / querysz
                accs["unlabel_query_nway"] = corrects["unlabel_query_nway"] / (unlabel_querysz)
            if self.gan:
                accs['gan_query_nway'] = corrects["gan_query_nway"] / (gan_qrysz)
        else:
            accs["query_nway"] = corrects["query_nway"] / querysz
        return accs

In [14]:
device = torch.device('cuda')
maml = Meta(args, config).to(device)

In [15]:
tmp = filter(lambda x: x.requires_grad, maml.parameters())
num = sum(map(lambda x: np.prod(x.shape), tmp))

In [16]:
path = args["save_path"]
step = 0
mkdir_p(path)
for epoch in range(args["epoch"]//6000):
        # fetch meta_batchsz num of episode each time

    train_dataloader = DataLoader(train_data_generator, args["task_num"], shuffle=True, num_workers=1, pin_memory=True)
    x_spt = y_spt = x_qry = y_qry = unlabel_spt = unlabel_qry = gan_qry = gan_spt = 0
    for idx,data  in enumerate(train_dataloader):
        if args["gan"]:
            with torch.no_grad():
                gan_spt_noise = torch.randn(args["task_num"]*args["spy_gan_num"], 100, 1, 1, device=device)
                gan_spt = netG(gan_spt_noise).to(device).view(args["task_num"],args["spy_gan_num"],args["img_c"],args["img_sz"],args["img_sz"])
                gan_qry_noise = torch.randn(args["task_num"]*args["qry_gan_num"], 100, 1, 1, device=device)
                gan_qry = netG(gan_qry_noise).to(device).view(args["task_num"],args["qry_gan_num"],args["img_c"],args["img_sz"],args["img_sz"])
        if len(data) == 4:
            (x_spt, y_spt, x_qry, y_qry) = data
            x_spt, y_spt, x_qry, y_qry = x_spt.to(device), y_spt.to(device), x_qry.to(device), y_qry.to(device)
            if args["gan"]:
                accs,loss_q = maml(x_spt, y_spt, x_qry, y_qry,gan_spt=gan_spt, gan_qry=gan_qry)
            else:
                accs,loss_q = maml(x_spt, y_spt, x_qry, y_qry)
        else:
            (x_spt, y_spt, x_qry, y_qry, unlabel_spt, unlabel_qry) = data
            x_spt, y_spt, x_qry, y_qry, unlabel_spt, unlabel_qry = x_spt.to(device), y_spt.to(device), x_qry.to(device), y_qry.to(device), \
            unlabel_spt.to(device), unlabel_qry.to(device)
            if args["spy_gan_num"]:
                accs,loss_q = maml(x_spt, y_spt, x_qry, y_qry,unlabel_spt_image=unlabel_spt, unlabel_qry_image=unlabel_qry,gan_spt=gan_spt, gan_qry=gan_qry)
            else:
                accs,loss_q = maml(x_spt, y_spt, x_qry, y_qry,unlabel_spt_image=unlabel_spt, unlabel_qry_image=unlabel_qry)

        writer.add_scalar('Loss/train_loss', loss_q, step)
        if "total_query_nway" in accs:
            writer.add_scalar('Accuracy/train_total_query_nway_accuracy', accs["total_query_nway"][-1], step)
        if "label_query_nway" in accs:
            writer.add_scalar('Accuracy/train_label_query_nway_accuracy', accs["label_query_nway"][-1], step)
            writer.add_scalar('Accuracy/train_unlabel_query_nway_accuracy', accs["unlabel_query_nway"][-1], step)
        if "gan_query_nway" in accs:
            writer.add_scalar('Accuracy/train_gan_query_nway_accuracy', accs["gan_query_nway"][-1], step)
        if "query_nway" in accs:
            writer.add_scalar('Accuracy/train_query_nway_accuracy', accs["query_nway"][-1], step)
        if step % 30 == 0:
            print('step:', step, '\ttraining acc:', accs)
        if step % 100 == 0:  # evaluation
            db_test = DataLoader(test_data_generator, 1, shuffle=True, num_workers=1, pin_memory=True)
            accs_all_test = {
                            "total_query_nway":[],
                            "unlabel_query_nway":[],
                            "query_nway":[],
                            "label_query_nway":[],
                            "gan_query_nway":[]
            }

            for test_data in db_test:
                if args["gan"]:
                    gan_spt_noise = torch.randn(args["spy_gan_num"], 100, 1, 1, device=device)
                    gan_spt = netG(gan_spt_noise).to(device)
                    gan_qry_noise = torch.randn(args["qry_gan_num"], 100, 1, 1, device=device)
                    gan_qry = netG(gan_qry_noise).to(device)
                if len(test_data) == 4:
                    x_spt, y_spt, x_qry, y_qry = test_data
                    x_spt, y_spt, x_qry, y_qry = x_spt.squeeze(0).to(device), y_spt.squeeze(0).to(device), \
                                                 x_qry.squeeze(0).to(device), y_qry.squeeze(0).to(device)
                    if args["gan"]:
                        accs,loss_q = maml.finetunning(x_spt, y_spt, x_qry, y_qry,gan_spt=gan_spt, gan_qry=gan_qry)
                    else:
                        accs = maml.finetunning(x_spt, y_spt, x_qry, y_qry)
                    
                else:
                    x_spt, y_spt, x_qry, y_qry, unlabel_spt, unlabel_qry = test_data
                    x_spt, y_spt, x_qry, y_qry, unlabel_spt, unlabel_qry = x_spt.squeeze(0).to(device), y_spt.squeeze(0).to(device), \
                                                 x_qry.squeeze(0).to(device), y_qry.squeeze(0).to(device),\
                                                unlabel_spt.squeeze(0).to(device), unlabel_qry.squeeze(0).to(device)
                    if args["gan"]:
                        accs = maml.finetunning(x_spt, y_spt, x_qry, y_qry, unlabel_spt, unlabel_qry,gan_spt=gan_spt, gan_qry=gan_qry)
                    else:
                        accs = maml.finetunning(x_spt, y_spt, x_qry, y_qry, unlabel_spt, unlabel_qry)
                if "total_query_nway" in accs:
                    accs_all_test["total_query_nway"].append(accs["total_query_nway"])
                if "label_query_nway" in accs:
                    accs_all_test["label_query_nway"].append(accs["label_query_nway"])
                    accs_all_test["unlabel_query_nway"].append(accs["unlabel_query_nway"])
                if "gan_query_nway" in accs:
                    accs_all_test["gan_query_nway"].append(accs["gan_query_nway"])
                if "query_nway" in accs:
                    accs_all_test["query_nway"].append(accs["query_nway"])
            # [b, update_step+1]
            if "total_query_nway" in accs:
                accs["total_query_nway"] = np.array(accs_all_test["total_query_nway"]).mean(axis=0).astype(np.float16)
                writer.add_scalar('Accuracy/test_total_query_nway_accuracy', accs["total_query_nway"][-1], step)
            if "label_query_nway" in accs:
                accs["label_query_nway"] = np.array(accs_all_test["label_query_nway"]).mean(axis=0).astype(np.float16)
                accs["unlabel_query_nway"] = np.array(accs_all_test["unlabel_query_nway"]).mean(axis=0).astype(np.float16)
                writer.add_scalar('Accuracy/test_label_query_nway_accuracy', accs["label_query_nway"][-1], step)
                writer.add_scalar('Accuracy/test_unlabel_query_nway_accuracy', accs["unlabel_query_nway"][-1], step)
            if "gan_query_nway" in accs:
                accs["gan_query_nway"] = np.array(accs_all_test["gan_query_nway"]).mean(axis=0).astype(np.float16)
                writer.add_scalar('Accuracy/test_gan_query_nway_accuracy', accs["gan_query_nway"][-1], step)
            if "query_nway" in accs:
                accs["query_nway"] = np.array(accs_all_test["query_nway"]).mean(axis=0).astype(np.float16)
                writer.add_scalar('Accuracy/test_query_nway_accuracy', accs["query_nway"][-1], step)
            
            print(idx, 'Test acc:', accs)


            torch.save(maml.state_dict(), "maml/" + path + "/model_step" + str(step) + ".pt")
        step += 1

step: 0 	training acc: {'total_query_nway': array([0.15555556, 0.23777778, 0.28888889, 0.3       , 0.28888889,
       0.29111111]), 'label_query_nway': array([0.152     , 0.18933333, 0.26933333, 0.27466667, 0.256     ,
       0.25866667]), 'query_nway': array([0.20533333, 0.26933333, 0.312     , 0.328     , 0.32      ,
       0.336     ]), 'unlabel_query_nway': array([0.2 , 0.48, 0.34, 0.42, 0.44, 0.44]), 'gan_query_nway': array([0.12, 0.48, 0.48, 0.44, 0.48, 0.48])}
0 Test acc: {'total_query_nway': array([0.1505, 0.2932, 0.3188, 0.327 , 0.3313, 0.337 , 0.3398, 0.3406,
       0.3416, 0.3435, 0.3445], dtype=float16), 'label_query_nway': array([0.1482, 0.26  , 0.2913, 0.2988, 0.3032, 0.3096, 0.312 , 0.3125,
       0.3137, 0.3162, 0.317 ], dtype=float16), 'query_nway': array([0.1888, 0.3464, 0.3843, 0.394 , 0.4001, 0.4062, 0.409 , 0.4102,
       0.411 , 0.4143, 0.4153], dtype=float16), 'unlabel_query_nway': array([0.2013, 0.5073, 0.5186, 0.535 , 0.5425, 0.545 , 0.5537, 0.5576,
       0.56

KeyboardInterrupt: 

In [ ]:
step